In [1]:
from google.colab import drive
drive.mount('/content/drive')

!cp -r /content/drive/MyDrive/LabSessionsIPCV/dataset.zip ./
!unzip dataset.zip

Mounted at /content/drive
Archive:  dataset.zip
   creating: dataset/
  inflating: __MACOSX/._dataset      
   creating: dataset/scenes/
  inflating: __MACOSX/dataset/._scenes  
  inflating: dataset/.DS_Store       
  inflating: __MACOSX/dataset/._.DS_Store  
   creating: dataset/models/
  inflating: __MACOSX/dataset/._models  
  inflating: dataset/scenes/scene12.png  
  inflating: __MACOSX/dataset/scenes/._scene12.png  
  inflating: dataset/scenes/scene10.png  
  inflating: __MACOSX/dataset/scenes/._scene10.png  
  inflating: dataset/scenes/scene11.png  
  inflating: __MACOSX/dataset/scenes/._scene11.png  
  inflating: dataset/scenes/scene5.png  
  inflating: __MACOSX/dataset/scenes/._scene5.png  
  inflating: dataset/scenes/scene4.png  
  inflating: __MACOSX/dataset/scenes/._scene4.png  
  inflating: dataset/scenes/scene6.png  
  inflating: __MACOSX/dataset/scenes/._scene6.png  
  inflating: dataset/scenes/scene7.png  
  inflating: __MACOSX/dataset/scenes/._scene7.png  
  inflating: 

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

# Paths to dataset directories
dataset_folder = 'dataset'
models_folder = os.path.join(dataset_folder, 'models')
scenes_folder = os.path.join(dataset_folder, 'scenes')

# Load images
def load_images(folder, prefix, count):
    images = []
    for i in range(1, count + 1):
        path = os.path.join(folder, f'{prefix}{i}.png')
        image = cv2.imread(path)
        if image is not None:
            images.append(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    return images

ref_images = load_images(models_folder, 'ref', 14)
scene_images = load_images(scenes_folder, 'scene', 5)

In [ ]:
# Initialize SIFT detector
sift = cv2.SIFT_create(nfeatures=5000)

# Function to detect and match features using SIFT and FLANN
def detect_and_match_features(ref_image, scene_image):
    # Detect and compute SIFT features
    kp1, des1 = sift.detectAndCompute(ref_image, None)
    kp2, des2 = sift.detectAndCompute(scene_image, None)

    # FLANN based matcher
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=100)

    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1, des2, k=2)

    # Apply ratio test
    good_matches = []
    for m, n in matches:
        if m.distance < 0.5 * n.distance:
            good_matches.append(m)

    # Extract matched keypoints
    points1 = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 2)
    points2 = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 2)

    # Apply RANSAC to filter out outliers
    if len(points1) >= 4:  # Minimum number of points needed to compute homography
        H, mask = cv2.findHomography(points1, points2, cv2.RANSAC, 3.0)
        matches_mask = mask.ravel().tolist()
    else:
        H, matches_mask = None, []

    return kp1, kp2, good_matches, matches_mask, H

# Function to find bounding boxes for matched features
def find_bounding_boxes(ref_image, scene_image, H):
    h, w = ref_image.shape[:2]

    # Get bounding box corners
    corners = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1, 1, 2)

    if H is not None:
        transformed_corners = cv2.perspectiveTransform(corners, H)
    else:
        transformed_corners = np.zeros((4, 1, 2), dtype=np.float32)

    return transformed_corners

# Function to check if bounding box contains sufficient context
def is_valid_bounding_box(bbox, scene_image):
    x_coords = bbox[:, 0, 0]
    y_coords = bbox[:, 0, 1]

    if (np.min(x_coords) >= 0 and np.max(x_coords) < scene_image.shape[1] and
        np.min(y_coords) >= 0 and np.max(y_coords) < scene_image.shape[0]):
        area = cv2.contourArea(bbox)
        return area > 5000  # Minimum area threshold
    return False

# Process each reference image against each scene image
for i, ref_image in enumerate(ref_images, 1):
    print(f'Processing Reference Image {i}')
    for j, scene_image in enumerate(scene_images, 1):
        kp1, kp2, good_matches, matches_mask, H = detect_and_match_features(ref_image, scene_image)

        bounding_box_valid = False
        if len(good_matches) >= 10 and H is not None:
            bounding_box = find_bounding_boxes(ref_image, scene_image, H)
            bounding_box_valid = is_valid_bounding_box(bounding_box, scene_image)

        # If not enough matches or invalid bounding box, prepare a dummy output
        if not bounding_box_valid:
            bounding_box = np.array([[[0, 0]], [[0, 0]], [[0, 0]], [[0, 0]]])
            print(f'Invalid bounding box or not enough matches for Scene {j} with Reference Image {i}')

        # Draw bounding box on scene image
        scene_image_with_box = scene_image.copy()
        bounding_box = np.int32(bounding_box)
        cv2.polylines(scene_image_with_box, [bounding_box], True, (0, 255, 0), 3, cv2.LINE_AA)

        # Draw matched keypoints
        draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                           singlePointColor = None,
                           matchesMask = matches_mask if bounding_box_valid else None, # draw only inliers
                           flags = 2)
        img_matches = cv2.drawMatches(ref_image, kp1, scene_image, kp2, good_matches, None, **draw_params)

        # Display matches and bounding box
        plt.figure(figsize=(20, 10))

        plt.subplot(1, 2, 1)
        plt.imshow(img_matches)
        plt.title(f'Matches for Scene {j}')
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.imshow(scene_image_with_box)
        plt.title(f'Bounding Box for Scene {j}')
        plt.axis('off')

        plt.show()